<a href="https://colab.research.google.com/github/SepKeyPro/genAI/blob/main/PyomoCodeQwen1_5_7B_Chat_PlayGround_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -U transformers datasets accelerate torch langchain langchain-mistralai langchain_core langchain_experimental

In [2]:
from datasets import load_dataset

from langchain_mistralai.chat_models import ChatMistralAI
import torch
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
)
from huggingface_hub import login
from google.colab import userdata

In [3]:
base_model = "SepKeyPro/PyomoCodeQwen1.5-7B-Chat" #"Qwen/Qwen2-72B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype="auto",
    device_map="auto"
)

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.63G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/237k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

# Model Inference

In [13]:
  dataset = load_dataset("SepKeyPro/PyomoRaw-200", split="train")
  new_dataset = dataset.train_test_split(test_size=0.1, shuffle=False)
  test_dataset = new_dataset["test"] # Total 22 examples in test dataset

  question = test_dataset[17]['Question']


  prompt = f"""Answer the following question about adding a constraint in Pyomo. Only generate the constraint. I don't need reasoning.
  Example: model.constraint = Constraint(expr=model.x > 0)
  Question: {question}
  Answer:"""

  messages = [
    {"role": "system", "content": "You are a helpful code assistant."},
    {"role": "user", "content": prompt},
  ]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
  )
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)
  result = pipe(text)
  output = result[0]['generated_text']
  response=""
  lines = output.splitlines()
  for i, line in enumerate(lines):
      if not line.startswith('Example:') and 'Constraint' in line:
          response = lines[i]
  print(f"Question: {question}, Answer: {response}")

Question: How do you add a constraint ensuring exp(x) + y^3 + z >= 7 in Pyomo?, Answer: model.constraint = Constraint(expr=exp(model.x) + model.y**3 + model.z >= 7)


In [14]:
code_template = """
from pyomo.environ import *
model = ConcreteModel()
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)
model.obj = Objective(expr=2 * model.x + 3 * model.y, sense=maximize)

# Define constraints
model.constraint1 = Constraint(expr=model.x + 2 * model.y <= 8)
model.constraint2 = Constraint(expr=3 * model.x + 2 * model.y <= 12)
[INSERT HERE]

# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model)
"""

constraint_to_insert = response

q = f"""
Code template: "{code_template}"
Constraint: "{constraint_to_insert}"
Please replace [INSERT_HERE] in Code template with the Constraint. Keep the order of the constraints with a correct number.
"""


In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
system_message = SystemMessage(content="You are an expert writer")
human_message = HumanMessage(content=q)
token = userdata.get("MISTRALAI")
# You may need to generate a token on MistralAI at https://console.mistral.ai/api-keys/
chat = ChatMistralAI(api_key=token)

res = chat.invoke([system_message, human_message])
res.pretty_print()

================================== Ai Message ==================================

Sure, I can help you with that. Here's the updated code template with the new constraint:

```python
from pyomo.environ import *
model = ConcreteModel()
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)
# Add a new variable z
model.z = Var(domain=NonNegativeReals)

model.obj = Objective(expr=2 * model.x + 3 * model.y, sense=maximize)

# Define constraints
model.constraint1 = Constraint(expr=model.x + 2 * model.y <= 8)
model.constraint2 = Constraint(expr=3 * model.x + 2 * model.y <= 12)
# Add the new constraint
model.constraint3 = Constraint(expr=exp(model.x) + model.y**3 + model.z >= 7)

# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model)
```

In this updated code template, I added a new variable `model.z` with a domain of `NonNegativeReals`. I then added the new constraint `model.constraint3` with the expression `exp(model.x) + model.y**3 + model.z >=